<a href="https://colab.research.google.com/github/diegoalvzfdez/master-data-science/blob/master/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

#Obtenemos los datos guardados en la nube de Google Drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/TFM\ Data\ Science/Gonzalo\ Velarde

/content/drive/My Drive/TFM Data Science/Gonzalo Velarde


In [0]:
#leemos el fichero
df = pd.read_csv('Velarde Rodriguez_Gonzalo_OD_31052017_120629_ELE.CSV', sep = ';', index_col= 0, header = 0, encoding = "ISO-8859-1")

Tenemos un fichero con la siguiente forma:

![](https://drive.google.com/uc?export=view&id=15UODzawnIyQqmrKIcTLwNOBOkcjqJh28)

Lo que tenemos es una representación de la profundidad de un ojo (eje z) en las coordenadas x (primera columna) y coordenadas y (primera fila)

Para poder manejar estos datos, en primer lugar buscaremos una manera de poder tener los datos más significativos de la manera más comoda para introducirlos en un clasificador.

En esta zona apuntaré las ideas que se me vayan ocurriendo para tratar dicha problemática (esperemos que no tener que escribir muchas):

1. Conseguir un vector de los n puntos con mayor profundidad: para ello vamos a tener que stackear los datos para conseguir todos los valores en una misma columna para después ordenarlos. (descartada)

2. Desechar la idea de introducir un vector de puntos e intentar definir el ojo del paciente mediante parámetros que nos permitan entrenar un sistema de clasificación. Si introducimos una gran cantidad de parámetros, generaremos una gran cantidad de ruido innecesario que nos impedirá entrenar el sistema de una manera eficiente.

# Idea 2

Para este caso vamos a intentar definir los parámetros más característicos que nos permitan entrenar el sistema de clasificación de la manera más eficiente. Los valores que vamos a definir son:

- Punto de máxima elevación
- Máxima curvatura (esta está definida mediante el radio de curvatura, por lo que a menor radio, mayor curvatura, por lo que nos quedaremos con el menor valor)
- Máxima desviación de la curvatura (en un ojo sano, la diferencia entre la menor curvatura y la mayor curvatura es pequeña, siendo en el caso de un ojo con queratocono más grande y por lo tanto un parámetro más a estudiar)
- Media de la elevación del ojo (en un ojo sano, la media de la elevación va a ser menor que en el caso de un ojo con queratocono, ya que existen valores más elevados. Además nos da una referencia de la distribución de la elevación de todo el ojo, sin estar definido unicamente por la máxima elevación)
- Media de la curvatura del ojo (en un ojo sano, esta media debe de estar casi siempre dentro de unos límites. En el caso de un ojo con queratocono, esta media nos permitirá definir como se distrubuyen las curvaturas en dicho ojo)
- Edad
- K máxima
- Paqui más fina


In [0]:
#Generamos los vectores que nos permitirán entrenar nuestro futuro sistema de clasificación

x_train = []
y_train = []

In [0]:
#generamos los parámetros necesarios para el fichero de elevación

df_ele = pd.read_csv('Velarde Rodriguez_Gonzalo_OD_31052017_120629_ELE.CSV', sep = ';', index_col= 0, header = 0, encoding = "ISO-8859-1")

In [6]:
#Stackeamos los datos para obtener las coordenadas de cada punto
df_ele_stacked = df_ele.stack()
df_ele_stacked = df_ele_stacked.reset_index()
df_ele_stacked = df_ele_stacked.rename(index = str, columns = {'FRONT': 'X', 'level_1': 'Y', 0: 'Z'})
#necesitamos una columna que simule el índice del dataframe
df_ele_stacked['ID'] = df_ele_stacked.index
df_ele_stacked['ID'] = pd.to_numeric(df_ele_stacked['ID'])
df_ele_stacked.head()

,X,Y,Z,ID
0,"5,200","-0,900",1967,0
1,"5,200","-0,800",1954,1
2,"5,200","-0,700",1943,2
3,"5,200","-0,600",1933,3
4,"5,200","-0,500",1925,4


In [0]:
df_ele_stacked_front = df_ele_stacked[df_ele_stacked['ID'] < list(df_ele_stacked[df_ele_stacked['X'] == 'BACK']['ID'])[0]]
df_ele_stacked_front.tail()

,X,Y,Z,ID
9973,"-5,900","0,100",2650,9973
9974,"-5,900","0,200",2653,9974
9975,"-5,900","0,300",2658,9975
9976,"-5,900","0,400",2665,9976
9977,"-5,900","0,500",2674,9977


In [0]:
#Borramos las columnas que no nos son necesarias
df_ele_stacked_front.drop(['X', 'Y', 'ID'], axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
#obtenemos el valor máximo de profundidad del ojo

df_ele_stacked_front['Z'] = pd.to_numeric(df_ele_stacked_front['Z'])
df_ele_stacked_front.sort_values('Z', ascending=False, inplace = True)
maxima_profundidad = df_ele_stacked_front['Z'][0]
print (maxima_profundidad)

2687


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#una vez obtenido el máximo valor de profundidad, modificamos el valor de todos los puntos para tenerlos obtener la bóveda sin invertir

df_ele_stacked_front['Z'] = (1 - (df_ele_stacked_front['Z']/maxima_profundidad))*maxima_profundidad
df_ele_stacked_front['Z'] = pd.to_numeric(df_ele_stacked_front['Z'])
df_ele_stacked_front.sort_values('Z', ascending=False, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
#Obtenemos el punto de máxima elevación

df_ele_stacked_front['Z'].max()

2687.0

In [0]:
x_train = []
x_train_v = []

In [0]:
#añadimos dicho valor al vector x_train_v (vector virtual que nos permitirá añadir valores al vector de entrenamiento)
x_train_v = []
x_train_v.append(int(df_ele_stacked_front['Z'].max()))

In [0]:
#para obtener los valores que definen la curvatura, debemos importar el fichero de curvatura

df_cur = pd.read_csv('Velarde Rodriguez_Gonzalo_OD_31052017_120629_CUR.CSV', sep = ';', index_col= 0, header = 0, encoding = "ISO-8859-1")

In [0]:
#realizamos las mismas transformaciones que con el fichero de elevación para poder obtener los valores que necesitamos

#Stackeamos los datos para obtener las coordenadas de cada punto
df_cur_stacked = df_cur.stack()
df_cur_stacked = df_cur_stacked.reset_index()
df_cur_stacked = df_cur_stacked.rename(index = str, columns = {'FRONT': 'X', 'level_1': 'Y', 0: 'Z'})
#necesitamos una columna que simule el índice del dataframe
df_cur_stacked['ID'] = df_cur_stacked.index
df_cur_stacked['ID'] = pd.to_numeric(df_cur_stacked['ID'])
df_cur_stacked.head()

,X,Y,Z,ID
0,"5,200","-0,900","8,27",0
1,"5,200","-0,800","8,25",1
2,"5,200","-0,700","8,24",2
3,"5,200","-0,600","8,22",3
4,"5,200","-0,500","8,21",4


In [0]:
df_cur_stacked_front = df_cur_stacked[df_cur_stacked['ID'] < list(df_cur_stacked[df_cur_stacked['X'] == 'BACK']['ID'])[0]]
df_cur_stacked_front.tail()

,X,Y,Z,ID
9973,"-5,900","0,100","7,98",9973
9974,"-5,900","0,200","7,98",9974
9975,"-5,900","0,300","7,98",9975
9976,"-5,900","0,400","7,98",9976
9977,"-5,900","0,500","7,98",9977


In [0]:
#Borramos las columnas que no nos son necesarias
df_cur_stacked_front.drop(['X', 'Y', 'ID'], axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
df_cur_stacked_front['Z'] = df_cur_stacked_front['Z'].apply(lambda x: x.replace(',', '.'))
df_cur_stacked_front['Z'] = pd.to_numeric(df_cur_stacked_front['Z'])
df_cur_stacked_front.sort_values('Z', ascending=True, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

In [0]:
#teniendo ya tratado el fichero de elevación, ya podemos introducir los valores necesarios para los parámetros de elevación

#introducimos el valor de maxima curvatura
x_train_v.append(float(df_cur_stacked_front.min()))

In [0]:
#introducimos el valor de la mayor desviación de la curvatura

x_train_v.append(float(df_cur_stacked_front.max()) - float(df_cur_stacked_front.min()))

In [0]:
#introducimos el valor de la media de las elevaciones

x_train_v.append(float(df_ele_stacked_front['Z'].mean()))

In [0]:
#introducimos el valor de la media de las curvaturas

x_train_v.append(float(df_cur_stacked_front['Z'].mean()))

In [0]:
#Obtenemos la edad del usuario

fecha_de_nacimiento = df_ele_stacked[df_ele_stacked['X'] == 'DateOfBirth']['Z'].get_values()[0]

In [0]:
#convertimos la fecha a formato fecha

import datetime

fecha_de_nacimiento = datetime.datetime.strptime(fecha_de_nacimiento, '%d-%m-%Y')

In [0]:
#Introducimos al vector la edad del paciente

x_train_v.append(2019 - int(fecha_de_nacimiento.year))

In [0]:
x_train_v

[2687, 7.85, 1.42, 1585.9665263579875, 8.069427741030163, 30]

In [0]:
#Introducimos la k máxima

k_max = df_ele_stacked[df_ele_stacked['X'] == 'K Max (Front)']['Z'].get_values()[0]

In [0]:
k_max = k_max.replace(',','.')

False

In [0]:
x_train_v.append(float(k_max))

In [0]:
x_train_v

[2687, 7.85, 1.42, 1585.9665263579875, 8.069427741030163, 30, 42.98]

In [0]:
#Introducimos la paqui más fina

paqui_min = df_ele_stacked[df_ele_stacked['X'] == 'Pachy Min']['Z'].get_values()[0]

In [0]:
x_train_v.append(int(paqui_min))

In [0]:
x_train_v

[2687, 7.85, 1.42, 1585.9665263579875, 8.069427741030163, 30, 42.98, 471]

In [12]:
df_ele_stacked_back = df_ele_stacked[df_ele_stacked['ID'] > list(df_ele_stacked[df_ele_stacked['X'] == 'BACK']['ID'])[0]]
df_ele_stacked_back = df_ele_stacked[df_ele_stacked['ID'] < list(df_ele_stacked[df_ele_stacked['X'] == 'BACK related to FRONT']['ID'])[0]]
df_ele_stacked_back.head()

,X,Y,Z,ID
0,"5,200","-0,900",1967,0
1,"5,200","-0,800",1954,1
2,"5,200","-0,700",1943,2
3,"5,200","-0,600",1933,3
4,"5,200","-0,500",1925,4
